In [2]:
#Imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from PIL import Image
import clip
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt



In [4]:
data = pd.read_csv('cat_dog1.csv')
data

,Unnamed: 0,image,labels
0,0,dog.624.jpg,1
1,1,cat.170.jpg,0
2,2,dog.419.jpg,1
3,3,dog.19.jpg,1
4,4,cat.227.jpg,0
...,...,...,...
1995,1995,dog.481.jpg,1
1996,1996,dog.159.jpg,1
1997,1997,cat.576.jpg,0
1998,1998,dog.190.jpg,1


In [3]:
#a-Spliting the dataset into training and testing sets with 20% for test dataset
#made it 0.9 for ease of testing for now
train_data, test_data = train_test_split(data, test_size=0.9, random_state=42)
X-train = train_data['image']
y-train = train_data['labels']
X-test = test_data['image']
y-test = test_data['labels']

In [16]:
# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"

model, transform = clip.load("ViT-B/32", 'cpu')

In [17]:
preprocessed_train = []
preprocessed_test = []

In [18]:
for file in X-train:
  preprocessed_train.append(transform(Image.open('cat_dog1/' + file )).unsqueeze(0).to(device))

for file in X-test:
  preprocessed_test.append(transform(Image.open('cat_dog1/' + file)).unsqueeze(0).to(device))

In [ ]:
#input_img = torch.tensor(np.stack(preprocessed_train)).cpu()
#i don't think leeha lazma but i'm leaving it here

In [19]:
#encode the inputs
image_embeddings_train = []
image_embeddings_test = []
for image in preprocessed_train:
  image_embeddings_train.append(model.encode_image(image))

for image in preprocessed_test:
  image_embeddings_test.append(model.encode_image(image))


In [14]:
image_embeddings_train = np.vstack(image_embeddings_train)
image_embeddings_test = np.vstack(image_embeddings_test)

array([[[ 1.0217872e-01,  2.8104293e-01,  1.0210094e-01, ...,
          9.5321190e-01,  3.7617832e-02, -1.2722777e-01]],

       [[-5.4830801e-01,  9.5876083e-02, -4.1318387e-02, ...,
          7.5573874e-01,  8.7017268e-03,  1.3450854e-01]],

       [[-2.5290810e-03,  1.0958649e-03, -1.8320274e-01, ...,
          1.0545205e+00,  2.4233526e-01,  2.7901149e-01]],

       ...,

       [[-2.8213200e-01,  6.0453430e-02, -1.1066309e-01, ...,
          1.1216140e+00,  2.4346091e-02,  3.8230765e-01]],

       [[-2.2695100e-01, -1.9104697e-01,  1.3683960e-01, ...,
          1.0613921e+00, -1.4923377e-01,  1.2921879e-01]],

       [[-1.2724274e-01,  1.0999836e-01,  1.4642596e-01, ...,
          1.0680327e+00,  1.6129503e-01,  3.9924258e-01]]], dtype=float32)

In [15]:
#Fisher's Algorithm
c = 0.1
means = [image_embeddings_train[train_data['labels'] == label].mean(axis=0) for label in [0, 1]]
S =  np.var(image_embeddings_train[train_data['labels'] == 0]) + np.var(image_embeddings_train[train_data['labels'] == 1])
S_inv = np.linalg.inv(S)
mean_dif = means[1] - means[0]
w = c*np.dot(S_inv, mean_dif)

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [ ]:
#predictions with c = 0.1
predict_c = np.dot(image_embeddings_test, w.T)

In [ ]:
#Scores
#Convert prediction scores into binary class predictions using a 0.0 threshold
predicted_labels = (predict_c > 0.0).astype(int)

#Calculations of metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

#Generating Confusion Matrix
confusion_matrix = confusion_matrix(true_labels, predicted_labels)
sns.heatmap(confusion_matrix, annot=True, fmt='d')
plt.title('Confusion Matrix')
plt.ylabel('Actual labels')
plt.xlabel('Predicted labels')
plt.show()

In [ ]:
#Bonus

C = [0.2, 0.3, 0.4, 0.5, 0.7]
W = []
for c in C:
  W.append(c*np.dot(S_inv, mean_dif))



In [ ]:
predict_C = []
for w in W:
  predict_C.append(np.dot(image_embeddings_test, w.T))

In [ ]:
#Scores
#Loop through each of the c values to calculate and print metrics for each
for i, c in enumerate(C):
    predicted_labels = (predict_C[i] > 0.0).astype(int)
    
    #Calculations of metrics
    accuracyB = accuracy_score(true_labels, predicted_labels)
    precisionB = precision_score(true_labels, predicted_labels)
    recallB = recall_score(true_labels, predicted_labels)
    f1B = f1_score(true_labels, predicted_labels)
    
    print(f'Metrics for c={c}:')
    print('Accuracy:', accuracyB)
    print('Precision:', precisionB)
    print('Recall:', recallB)
    print('F1 Score:', f1B)
    
#Generating Confusion Matrix
    confusion_matrixB = confusion_matrix(true_labels, predicted_labels)
    sns.heatmap(confusion_matrixB, annot=True, fmt='d')
    plt.title(f'Confusion Matrix for c={c}')
    plt.ylabel('Actual labels')
    plt.xlabel('Predicted labels')
    plt.show()
